In [1]:
import lightning as L
from model import BaseLineUnet
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from dataset import GoProDataset, AnimeDataset
from torchvision.transforms import v2
import matplotlib.pyplot as plt
import random
from NAFnet_baseline.Baseline_arch import Baseline
import torch.nn.functional as F

In [ ]:
# img_channel = 3
# width = 32

# dw_expand = 1
# ffn_expand = 2


# enc_blks = [1, 1, 1, 28]
# middle_blk_num = 1
# dec_blks = [1, 1, 1, 1]

# model = Baseline(img_channel=img_channel, width=width, middle_blk_num=middle_blk_num,
#                  enc_blk_nums=enc_blks, dec_blk_nums=dec_blks, dw_expand=dw_expand, ffn_expand=ffn_expand)
model = BaseLineUnet()
checkpoint = torch.load("lightning_logs/baselineunetwithlpipsloss2/checkpoints/best-checkpoint-epoch=55-train_loss=-103.956345.ckpt")

In [3]:
model_weights = checkpoint["state_dict"]
for key in list(model_weights):
    model_weights[key.replace("model.", "")] = model_weights.pop(key)

In [4]:
for key in list(model_weights):
    if key.startswith('loss_fn.'):
        model_weights.pop(key)

In [ ]:
model.load_state_dict(model_weights)
model.eval()

In [6]:
# transform = v2.Compose([
#     v2.ToImage(),
#     v2.ToDtype(torch.float32, scale=True)
# ])
# dataset = GoProDataset(transform=transform, size=512)
dataset = GoProDataset()

In [ ]:
sample = dataset[random.randint(0, len(dataset))]
with torch.inference_mode():
    predict = model(sample[0].unsqueeze(0))

fig, axes = plt.subplots(1, 3, figsize=(10, 5))
axes[0].imshow(sample[0].numpy().transpose(1, 2, 0))
axes[1].imshow(sample[1].numpy().transpose(1, 2, 0))
axes[2].imshow(predict.squeeze().numpy().transpose(1, 2, 0))
plt.savefig('result.png')
plt.show()

In [ ]:
F.mse_loss(sample[1], predict)

In [9]:
image = v2.ToPILImage()(torch.clamp(sample[0], min=0, max=1))
image.save('blur.png')
image = v2.ToPILImage()(torch.clamp(sample[1], min=0, max=1))
image.save('sharp.png')
image = v2.ToPILImage()(torch.clamp(predict.squeeze(), min=0, max=1))
image.save('predict.png')